# **Initiation**

In [1]:
# %pip install bs4
# %pip install html2text
# %pip install clean-text
# %pip install undetected-chromedriver requests
# %pip install selenium["4.18.1"]

# **Imports**

In [15]:
import pandas as pd
import bs4
from urllib.parse import urlparse, parse_qs
import urllib.parse
from selenium import webdriver as wd2
import undetected_chromedriver as webdriver
from cleantext import clean
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from itertools import product
import html2text
from selenium.webdriver.support import expected_conditions as EC
from transformers import MarkupLMFeatureExtractor, MarkupLMTokenizerFast, MarkupLMProcessor, MarkupLMModel
import os
import gc
pd.set_option('display.max_columns', None)
# from umap import UMAP
import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
import sweetviz as sv
from tqdm.notebook import tqdm
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import AgglomerativeClustering, DBSCAN, OPTICS, KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import numpy as np
import requests
from IPython.core.display import display, HTML
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# import accelerate
# from g4f.client import Client
import bs4
# import nltk
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize
# import nltk
import re
import json
# from googlesearch import search
from time import sleep
from tqdm import tqdm
from typing import List

# nltk.download('punkt')

/tmp/ipykernel_5762/410238637.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# **Values**

In [16]:
n_of_sources = 100
std_header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}


In [17]:
driver_path = './chromedriver'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--use_subprocess")
chrome_options.add_argument("javascript.enabled")
browser = webdriver.Chrome(driver_executable_path='./chromedriver', options=chrome_options)

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


In [18]:
wait = WebDriverWait(browser, 20)

In [19]:
sup_words = ['reveiws', 'descriptions', 'marks']

# Functions

In [7]:
def search_trust_pylot_by_category(category: str, supword: str='review', border: int=50, body='res'):
    t = urllib.parse.quote(category + ' ' + supword, safe='')
    v= f"https://www.google.com/search?channel=fs&client=ubuntu-sn&q={t}"
    browser.get(f"https://www.google.com/search?channel=fs&client=ubuntu-sn&q={t}")
    wait.until(EC.presence_of_element_located((By.ID, body)))
    r = browser.find_element(by=By.ID, value=body)
    x = []
    for i in r.find_elements(by=By.TAG_NAME, value='a'):
        x.append(i.get_attribute('href'))
    return x[:border]


In [8]:
def selenium_google_search_source(text: str, browser, border: int=20, body='rcnt'): # 'rcnt'
    t = urllib.parse.quote(text, safe='')
    v= f"https://www.google.com/search?channel=fs&client=ubuntu-sn&q={t}"
    browser.get(f"https://www.google.com/search?channel=fs&client=ubuntu-sn&q={t}")
    x = []
    for i in browser.find_element(by=By.ID, value=body).find_elements(by=By.TAG_NAME, value='a'):
        x.append(i.get_attribute('href'))
    return x[:border]


In [9]:
def selenium_source_link_to_html(links: List[str], browser):
    html_source = []
    bad_res = []
    for k, i in enumerate(links):
        try:
            browser.get(i)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            html_source.append(browser.page_source)
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            print('*', end='')
            bad_res.append(i)
        print(k, end=', ')
    return html_source, bad_res


def req_source_link_to_html(links: List[str], browser):
    html_source = []
    bad_res = []
    for k, i in enumerate(links):
        try:
            r = requests.get(i, headers=std_header)
            if r.status_code == 200:
                html_source.append(r.text)
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            # print('*', end='')
            bad_res.append(i)
        # print(k, end=', ')
    return html_source, bad_res



def req_source_link_to_html_saving(links: List[str], req_id, browser):
    html_source = []
    bad_res = []
    for k, i in enumerate(links):
        try:
            r = requests.get(i, headers=std_header, timeout=40)
            if r.status_code == 200:
                with open(f'./html_pages/{req_id}_{k}.html', 'w') as f:
                    f.write(r.text)
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            # print('*', end='')
            bad_res.append(i)
        # print(k, end=', ')
    return html_source, bad_res

In [10]:
def parse_requests(search_text: list[str], border=50):
    bad_res = []
    result_search = dict()
    for i, j in enumerate(search_text):
        try:
            sleep(1)
            x = selenium_google_search_source(j, browser=browser, border=border)
            result_search[j] = dict(enumerate(filter(lambda a: a != None, x)))
        except Exception as e:
            if type(e) is KeyboardInterrupt:
                break
            print('*', end='')
            bad_res.append(j)
        print(i, end=', ')

    return result_search, bad_res

# Get categories

In [24]:
r = requests.get('https://www.trustpilot.com/categories', headers=std_header).text
categories = bs4.BeautifulSoup(r, 'html.parser').find("section", class_='categories_categoriesWrapper__fIt_K').find_all('ul')

In [25]:
sub_categories = sum([[i.text for i in i.find_all('li')] for i in categories], [])

In [26]:
with open('./trust_pilot_cat.json', 'w') as f:
    json.dump({'subcategories': sub_categories}, f)

# Get Links

In [11]:
with open('./trust_pilot_cat.json', 'r') as f:
    categories = json.load(f)['subcategories']

In [12]:
len(categories) * 3

567

In [13]:
try: 
    with open('links_lists.json', 'r') as f:
        res = json.load(f)
except:
    res = dict()

In [14]:
len(res)

542

In [15]:
for c, sw in tqdm(product(categories, sup_words)):
    # sleep(1)
    k = c + ' '  + sw
    if (not k in res) or (list(res.keys())[-1] == k):
        res[k] = search_trust_pylot_by_category(c, sw)

567it [00:37, 15.19it/s] 


In [17]:
with open('links_lists.json', 'w') as f:
    json.dump(res, f)

In [18]:
with open('links_lists_formated.json', "+a", encoding = "utf-8") as file:
    json.dump(res, file, ensure_ascii = False)

# Parse Trust Pilot

In [19]:
from urllib.parse import urlparse

In [20]:
with open('links_lists.json', 'r') as f:
        res = json.load(f)

In [22]:
trustpilot_link_index_pair = dict()
for vs in res.values():
    for i, v in enumerate(vs):
        o = urlparse(v)
        if 'trustpilot' in str(o.netloc):
            trustpilot_link_index_pair[v] = i


In [24]:
with open('trustpilot_link_index_pair.json', "+a", encoding = "utf-8") as file:
    json.dump(trustpilot_link_index_pair, file, ensure_ascii = False)

# Get Trust_pilot_html

In [53]:
with open('./trustpilot_link_index_pair.json', 'r') as f:
    trustpilot_link_index_pair = json.load(f)

In [59]:
for i in tqdm(trustpilot_link_index_pair):
    o = urlparse(i)
    fn = f'{str(o.netloc.replace(".", "_"))}____{str(o.path).replace("/", "_")}____{str(o.query).replace("=", "_")}____{trustpilot_link_index_pair[i]}.html'
    r = requests.get(i, headers=std_header).text
    with open(f'tp_htmls/{fn}', 'w') as f:
        f.write(r)

100%|██████████| 218/218 [03:23<00:00,  1.07it/s]


# Embedding

In [20]:
processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
model = MarkupLMModel.from_pretrained("microsoft/markuplm-base")

In [22]:
def removetextbetweentags(html):
    soup = bs4.BeautifulSoup(html, 'html.parser')
    
    # Remove all text between nodes
    for element in soup.find_all(text=True):
        if isinstance(element, bs4.Comment):
            continue  # Skip comments
        element.replace_with('')
    
    # Remove contents of all script and style tags
    for script_or_style in soup(['script', 'style']):
        script_or_style.clear()
    
    return str(soup)

In [24]:
window = 1000
for fn in tqdm(os.listdir('./tp_htmls/')):
    with open(f'./tp_htmls/{fn}', 'r') as f:
        # r = bs4.BeautifulSoup(f.read(), 'html.parser')
        r = bs4.BeautifulSoup(removetextbetweentags(f.read()), 'html.parser')
        r = r.prettify()
        l = r.split('\n')
        # r_p = str(r)
        r_p = [''.join(l[i:i+window]) for i in range(0, len(l), window)]
    k = list(map(lambda a: model(**processor(a, return_tensors="pt")).last_hidden_state.squeeze(), r_p))
    outputs = torch.cat(k, 0)
    torch.save(outputs, f'./vectorized_torch_seq/{fn.replace("json", "")}.pt')
    del outputs
    # del l
    # del r_p
    gc.collect()

  0%|          | 0/218 [00:00<?, ?it/s]/tmp/ipykernel_5762/3579530471.py:5: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for element in soup.find_all(text=True):
100%|██████████| 218/218 [02:13<00:00,  1.63it/s]
